#### 0

In [17]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 

In [38]:
np.random.seed(42)

X = np.random.uniform(0, 1, size=(1000,2))
X.shape, X[0:5]

((1000, 2),
 array([[0.37454012, 0.95071431],
        [0.73199394, 0.59865848],
        [0.15601864, 0.15599452],
        [0.05808361, 0.86617615],
        [0.60111501, 0.70807258]]))

In [39]:
epsilon = np.random.normal(0, 1, size=1000)
epsilon.shape

(1000,)

In [41]:
# Terrible solution, but the dataset was terribly formatted to begin with
X1 = np.array([X[i][0] for i in range(len(X))])
X2 = np.array([X[i][1] for i in range(len(X))])

y = 3 * X1 + 5 * X2 + 3 + epsilon
y.shape

(1000,)

In [21]:
X = np.c_[np.ones(len(X)), X]
X[0:3], X.shape

(array([[1.        , 0.37454012, 0.95071431],
        [1.        , 0.73199394, 0.59865848],
        [1.        , 0.15601864, 0.15599452]]),
 (1000, 3))

#### 1 a)

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape # X has two columns, so shape is different

((670, 3), (330, 3), (670,), (330,))

In [23]:
y_train = y_train.reshape(-1, 1)

In [25]:
np.random.seed(42)

def gradient_descent(X, y, learning_rate=0.1, epochs=100):
    m = len(X)
    
    # Create random b0 & b1
    theta = np.random.randn(X.shape[1], 1)  # [beta_0, beta_1]^T - [2x1]
    
    for _ in range(epochs):
        # Get gradient (slope)
        gradient = 2 / m * X.T @ (X @ theta - y) # idk
        
        # determine step size and use it next time we loop
        theta -= learning_rate*gradient # learning_rate is eta in theory
    return theta

# Get estimated b0 and b1 values by actually doing the batch gradient descent
theta = gradient_descent(X_train, y_train, epochs=1000, learning_rate=0.1)

theta.reshape(-1) # throws away additional dimension





array([3.16440925, 3.03763668, 4.82471836])